In [30]:
from IPython.display import clear_output

In [31]:
class Card():
    def __init__(self, suit, face):
        self.suit = suit
        self.face=face
    
    def value(self):
        face=self.face
        if isinstance(face,int):
            return face
        elif face is 'J' or face is 'Q' or face is 'K':
            return 10
        elif face is 'A':
            return 11
        

In [32]:
from random import choice,shuffle
class Deck():
    def __init__(self,multiplier):
        self.stack=[]
        for i in range(2,11):
            self.stack.append(Card('Spade',i))
            self.stack.append(Card('Heart',i))
            self.stack.append(Card('Club',i))
            self.stack.append(Card('Diamond',i))
        for f in ['J','Q','K','A']:
            self.stack.append(Card('Spade',f))
            self.stack.append(Card('Heart',f))
            self.stack.append(Card('Club',f))
            self.stack.append(Card('Diamond',f))
        self.stack*=multiplier
        self.dealt=[]
        
    
    def shuffle(self):
        self.stack+=self.dealt
        self.dealt=[]
        shuffle(self.stack)
    
    def dealone(self):
        top_card=self.stack.pop()
        self.dealt.append(top_card)
        return top_card
    
        

In [33]:
class Player():
    def __init__(self,money):
        self.money=money
        self.hand=[]
        self.splithand=[]
        self.bet=0
        self.splitbet=0
        
        
    def handvalue(self,split=False):
        total=0
        aces=0
        if not split:
            the_hand=self.hand
        else:
            the_hand=self.splithand
        for h in the_hand:
            if h.face=='A':
                aces+=1
            total+=h.value()
            while total>21 and aces>0:
                total-=10
                aces-=1
        return total
            

    def wager(self,opp,value,split=False):
        if split:
            self.splitbet+=value
            opp.splitbet+=value
        else:
            self.bet+=value
            opp.bet+=value
        opp.money-=value
        self.money-=value
    
    def win(self,split=False):
        if split:
            self.money+=self.splitbet*2
            self.splitbet=0
        else:
            self.money+=self.bet*2
            self.bet=0
    
    def lose(self,split=False):
        if split:
            self.bet=0
        else:
            self.splitbet=0
    
    def tie(self,split=False):
        if split:
            self.money+=self.splitbet
            self.splitbet=0
        else:
            self.money+=self.bet
            self.bet=0
    
    def printhand(self,split=False):
        if split:
            the_hand=self.splithand
        else:
            the_hand=self.hand
        for h in the_hand:
            print("{} of {}s".format(h.face,h.suit))
    
            
    def printmoney(self):
        print("You have ${}".format(self.money))
    
        

In [34]:
class Dealer(Player):
    def __init__(self,money):
        super().__init__(money)
        self.money=money
        
    def musthit(self):
        if self.handvalue()<18:
            return True
        else:
            return False
    def printmoney(self):
        print("The dealer has ${} of your money".format(self.money))
    
    def printhand(self,pre=True):
        if pre:
            print("? of ?")
        else:
            print("{} of {}s".format(self.hand[0].face,self.hand[0].suit))
        for h in self.hand[1:]:
            print("{} of {}s".format(h.face,h.suit))

In [35]:
class Inputs():
    def __init__(self):
        self.start=False
        self.quit=False
        self.commands=['Hit', 'Double Down','Stay','Split','Q','Quit','Y','Yes','N','No']
        self.decision=""
        

    def setto(self,question):
        string='Blank'
        while string not in self.commands:
            string=input(question).title()
        if string.title()=='Y' or string.title()=='Yes':
            self.start=True
        else:
            self.start=False
        if string.title()=='Q' or string.title()=='Quit':
            self.quit=True
        else:
            self.quit=False
        self.decision=string

In [36]:
def print_intro():
    print("Welcome to BlackJack!!")
    print("======================")
    print("How to play:")
    print("")
    print("1) Have a higher hand value than the dealer to win.")
    print("2) Going over 21 is called a 'bust' and you lose automatically.")
    print("3) The dealer must hit on any hand lower than 17.")
    print("4) If you and the dealer tie, it's a 'push' and neither you nor the dealer loses money.")
    print("5) Type 'hit' to receive a card from the deck on your turn.")
    print("6) Type 'stay' to end your turn.")
    print("7) Type 'double' to double down (once doubling down, your turn ends after receiving card).")
    print("8) Type 'split' to split the cards in your hand (only allowed on pairs).")
    print("9) Each player begins with $500")
    print("10) The player with the highest amount during the session will be displayed when you leave the table.")
    print("11) Press 'q' to quit at any time.")
    print("\nGood Luck!!")

In [37]:
def inithands(bot,player,deck):
    bot.hand=[]
    bot.hand.append(deck.dealone())
    bot.hand.append(deck.dealone())
    player.hand=[]
    player.hand.append(deck.dealone())
    player.hand.append(deck.dealone())


In [38]:
def printinfo(bot,player,pre=True,split=False):
    clear_output()
    bot.printhand(pre)
    if not pre:
        print("Their hand is worth {}.".format(bot.handvalue()))
    print("================")
    player.printmoney()
    print("You bet ${}".format(player.bet))
    if split:
        print("You bet ${} on the split".format(player.splitbet))
    print("================")
    player.printhand()
    print('')
    print("This hand is worth {}.".format(player.handvalue()))
    if split:
        print('\n')
        player.printhand(split)
        print('')
        print("This hand is worth {}.".format(player.handvalue(split)))

In [39]:
def checkwinner(bot,player,split=False):    
    if bot.handvalue(split)>21 and player.handvalue(split)>21:
        print("You both busted.")
        print("It's a Push")
        player.tie(split)
        bot.tie(split)
    elif bot.handvalue(split)==player.handvalue(split):
        print("It's a Push.")
        player.tie()
        bot.tie()
    elif bot.handvalue(split)>21 and player.handvalue(split)<=21: 
        print("They busted.")
        print('You Win!')
        player.win(split)
        bot.lose(split)
    elif bot.handvalue(split)<=21 and player.handvalue(split)>21:          
        print("You busted.")
        print("You Lose...")
        bot.win(split)
        player.lose(split)            
    elif bot.handvalue(split)>player.handvalue(split):
        print('You Lose...')
        bot.win(split)
        player.lose(split)
    else:
        print('You Win!')
        player.win(split)
        bot.lose(split)

In [40]:
def the_game():
    print_intro()
    u_input=Inputs()
    u_input.setto("Would you like to play? [y/n] ")
    if not u_input.start:
        print('Have a nice day! ')
        return None
    else:
        #initialize the game
        money=input("How much money do you want to play with? ")
        if money.title()=="Q" or money.title()=='Quit':
            u_input.quit=True
        while not money.isdigit():
            clear_output()
            money=input("How much money do you want to play with? ")
            if money.title()=="Q" or money.title()=='Quit':
                u_input.quit=True
                break
        money=int(money)
        while money<1:
            clear_output()
            print("Are you sure that's enough?")
            money=input("How much money do you REALLY want to play with? ")
            while not money.isdigit():
                clear_output()
                money=input("How much money do you want to play with? ")
                if money.title()=="Q" or money.title()=='Quit':
                    u_input.quit=True
                    break
            if money.title()=="Q" or money.title()=='Quit':
                u_input.quit=True
                break
            money=int(money)
        dealer=Dealer(0)
        player1=Player(money)
        deck=Deck(7)
        deck.shuffle()
    
    #plays the hand
    while u_input.start==True and u_input.quit==False:
        if player1.money<1:
            print("You are out of money.")
            money=input("How much more money do you want to put down? [type '0' or 'q' to stop] ")
            if money.title()=="Q" or money.title()=='Quit' or money.title()=='0':
                u_input.quit=True
                continue
            while not money.isdigit():
                clear_output()
                print("You are out of money.")
                money=input("How much more money do you want to put down? [type '0' or 'q' to stop] ")
            if money.title()=="Q" or money.title()=='Quit' or money.title()=='0':
                if money.title()=="Q" or money.title()=='Quit':
                    u_input.quit=True
                    break
            money=int(money)
            player1.money+=money
        if str(money).title()=="Q" or str(money).title()=='Quit':
            u_input.quit=True
            continue
        clear_output()
        player1.printmoney()
        wager=input("How much money do you want to wager? ")
        if str(wager).title()=="Q" or str(wager).title()=='Quit':
            u_input.quit=True
            continue
        while not wager.isdigit():
            clear_output()
            player1.printmoney()
            wager=input("How much money do you want to wager? ")
            if wager.title()=="Q" or wager.title()=='Quit':
                u_input.quit=True
                break
        if str(wager).title()=="Q" or str(wager).title()=='Quit':
            u_input.quit=True
            continue
        wager=int(wager)
        while wager>player1.money or 1>wager:
            print("That isn't a valid amount!")
            player1.printmoney()
            wager=input("How much would you like to bet? ")
            while not wager.isdigit():
                clear_output()
                player1.printmoney()
                wager=input("How much money do you want to wager? ")
                if wager.title()=="Q" or wager.title()=='Quit':
                    u_input.quit=True
                    break
            if wager.title()=="Q" or wager.title()=='Quit':
                u_input.quit=True
                break
            wager=int(wager)
        if str(wager).title()=="Q" or str(wager).title()=='Quit':
            u_input.quit=True
            continue
        player1.wager(dealer,wager)
        inithands(dealer,player1,deck)
        clear_output()
        printinfo(dealer,player1)
        split=False
        if player1.hand[0].face==player1.hand[1].face:
            u_input.setto('You can Split, Double Down, Hit, or Stay')
        else:
            u_input.setto('You can Double Down, Hit, or Stay')
        if u_input.decision=="Q" or u_input.decision=='Quit':
            u_input.quit=True
            continue
            
            
        if u_input.decision=='Split':
            split=True
            player1.splithand=[player1.hand.pop()]
            clear_output()
            player1.hand.append(deck.dealone())
            player1.splithand.append(deck.dealone())
            player1.wager(dealer,wager,split=split)
            printinfo(dealer,player1,split=split)
            
            
        
        if split==True:
            u_input.setto('You can Hit, or Stay on the first ')
            while u_input.decision=='Hit':
                player1.splithand.append(deck.dealone())
                clear_output()
                printinfo(dealer,player1,split)
                if player1.handvalue(split)>21:
                    break
                else:
                    u_input.setto('You can Hit, or Stay ')
                    if u_input.decision=="Q" or u_input.decision=='Quit':
                        u_input.quit=True
                        break
            if u_input.decision=="Q" or u_input.decision=='Quit':
                u_input.quit=True
                continue      
            #second go around for the split hand
            u_input.setto('You can Hit, or Stay on the other hand ')
            while u_input.decision=='Hit':
                player1.hand.append(deck.dealone())
                clear_output()
                printinfo(dealer,player1,split)
                if player1.handvalue(split)>21:
                    break
                else:
                    u_input.setto('You can Hit, or Stay ')
                    if u_input.decision=="Q" or u_input.decision=='Quit':
                        u_input.quit=True
                        break
            if u_input.decision=="Q" or u_input.decision=='Quit':
                u_input.quit=True
                continue   
        
        if u_input.decision=='Hit' and split==False:
            while u_input.decision=='Hit':
                player1.hand.append(deck.dealone())
                clear_output()
                printinfo(dealer,player1)
                if player1.handvalue()>21:
                    break
                else:
                    u_input.setto('You can Hit, or Stay ')
                    if u_input.decision=="Q" or u_input.decision=='Quit':
                        u_input.quit=True
                        break
            if u_input.decision=="Q" or u_input.decision=='Quit':
                u_input.quit=True
                continue      
        
        elif u_input.decision=='Double Down' and split==False:
            player1.hand.append(deck.dealone())
            player1.wager(dealer,wager)
            printinfo(dealer,player1)
        while dealer.musthit():
            dealer.hand.append(deck.dealone())
        #Tell the player what happened in the background
        clear_output()
        printinfo(dealer,player1,pre=False,split=split)
        print('\n')
        if split:
            print("The first hand:")
            checkwinner(dealer,player1)
            print("The second hand:")
            checkwinner(dealer,player1,split)
        else:
            checkwinner(dealer,player1)

        u_input.setto('Would you like to continue? ')
    
    clear_output()
    if dealer.money>player1.money:
        print("The house took ${}".format(dealer.money))
    else:
        print("You walked away with ${}".format(player1.money))

In [41]:
the_game()

The house took $70
